# S09 — Timbre, enerxía e “pegada sonora” (features + comparación)
**Curso:** Programación Musical con Python — 2º Trimestre  
**Duración:** 1 sesión (≈ 1 hora)

## 🎯 Obxectivos
- Entender “features” sinxelas de audio: **enerxía** e **timbre**.
- Calcular e visualizar:
  - RMS (enerxía)
  - Centroide espectral (brillo)
  - Zero Crossing Rate (rugosidade/ruído)
- Construír unha **pegada sonora** (vector de features) para comparar audios.
- Mini-clasificación simple: decidir cal fragmento é A/B con regras.

> 🎛️ Hoxe facemos MIR suave: datos do son, sen IA complexa.


## 0) Setup (Colab)


In [ ]:
!pip -q install librosa soundfile

import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display

from IPython.display import Audio, display

print("✅ librosa listo")


## 1) Cargar audio(s)

Usaremos 2 audios de exemplo de librosa para comparar.
Tamén podes usar 1 audio e cortar 2 fragmentos (como no S08).


In [ ]:
path1 = librosa.ex("trumpet")
path2 = librosa.ex("choice")

y1, sr1 = librosa.load(path1, sr=None)
y2, sr2 = librosa.load(path2, sr=None)

print("Audio1 sr/dur:", sr1, round(len(y1)/sr1, 2), "s")
print("Audio2 sr/dur:", sr2, round(len(y2)/sr2, 2), "s")

display(Audio(y1, rate=sr1))
display(Audio(y2, rate=sr2))


## 2) Features básicas (a nosa “caixa de ferramentas”)

### RMS (Root Mean Square) = enerxía
- canto máis alto, máis “forte” (en media)

### Centroide espectral = brillo
- canto máis alto, máis “brillante” / máis agudos dominantes

### Zero Crossing Rate (ZCR)
- cantos cruces por cero: máis alto → máis ruído / textura áspera


In [ ]:
def features_basicas(y, sr):
    rms = librosa.feature.rms(y=y)[0]
    centroid = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
    zcr = librosa.feature.zero_crossing_rate(y)[0]
    return rms, centroid, zcr

rms1, cent1, zcr1 = features_basicas(y1, sr1)
rms2, cent2, zcr2 = features_basicas(y2, sr2)

print("Feature lengths:", len(rms1), len(cent1), len(zcr1))


## 3) Visualizar features no tempo

Debuxamos cada feature como unha curva ao longo do tempo.


In [ ]:
def plot_feature(feat, sr, hop_length=512, title=""):
    times = librosa.frames_to_time(np.arange(len(feat)), sr=sr, hop_length=hop_length)
    plt.figure()
    plt.plot(times, feat)
    plt.title(title)
    plt.xlabel("Tempo (s)")
    plt.show()

plot_feature(rms1, sr1, title="RMS (enerxía) - audio1")
plot_feature(cent1, sr1, title="Centroide espectral (brillo) - audio1")
plot_feature(zcr1, sr1, title="ZCR - audio1")


In [ ]:
plot_feature(rms2, sr2, title="RMS (enerxía) - audio2")
plot_feature(cent2, sr2, title="Centroide espectral (brillo) - audio2")
plot_feature(zcr2, sr2, title="ZCR - audio2")


## 4) “Pegada sonora” (fingerprint) simple

Imos resumir cada audio cun vector de 6 números:
- media e desviación típica de RMS
- media e desviación típica de centroide
- media e desviación típica de ZCR

Isto é unha pegada numérica para comparar.


In [ ]:
def fingerprint(y, sr):
    rms, cent, zcr = features_basicas(y, sr)
    vec = np.array([
        np.mean(rms),  np.std(rms),
        np.mean(cent), np.std(cent),
        np.mean(zcr),  np.std(zcr)
    ], dtype=float)
    return vec

fp1 = fingerprint(y1, sr1)
fp2 = fingerprint(y2, sr2)

fp1, fp2


### Comparación por distancia

Se dous audios teñen pegadas parecidas, a distancia será pequena.
Usaremos distancia euclídea (sinxela).


In [ ]:
def dist(a, b):
    return float(np.linalg.norm(a - b))

print("Distancia fp1–fp2:", round(dist(fp1, fp2), 4))


## 5) Mini-clasificador por regras (sen IA)

Exemplo de regra:
- Se o centroide medio é alto → máis brillante
- Se o ZCR medio é alto → máis ruídoso

Imos facer unha función que “describe” un audio.


In [ ]:
def describir(fp):
    rms_m, rms_s, cent_m, cent_s, zcr_m, zcr_s = fp
    desc = []
    desc.append("máis forte" if rms_m > 0.05 else "máis suave")
    desc.append("máis brillante" if cent_m > 2000 else "máis escuro")
    desc.append("máis ruídoso" if zcr_m > 0.08 else "máis limpo")
    return ", ".join(desc)

print("Audio1:", describir(fp1))
print("Audio2:", describir(fp2))


## 6) Clasificación simple de fragmentos (exercicio)

Imos cortar 2 fragmentos do mesmo audio e ver se podemos diferencialos.


In [ ]:
def cortar(y, sr, inicio_s, fin_s):
    i0 = int(inicio_s * sr)
    i1 = int(fin_s * sr)
    return y[i0:i1]

# Cortes do audio2 (rítmico)
fragA = cortar(y2, sr2, 0, 6)
fragB = cortar(y2, sr2, 6, 12)

display(Audio(fragA, rate=sr2))
display(Audio(fragB, rate=sr2))

fpA = fingerprint(fragA, sr2)
fpB = fingerprint(fragB, sr2)

print("FragA:", describir(fpA))
print("FragB:", describir(fpB))
print("Distancia A–B:", round(dist(fpA, fpB), 4))


## 🧩 Mini-proxecto avaliable (entrega)

### Parte A (técnica)
Escolle 2 opcións:

**Opción 1 (dous audios distintos)**
- Comparar 2 audios (ex. trompeta vs ritmo)
- Calcular RMS, centroide, ZCR
- Construír fingerprint e distancia

**Opción 2 (dous fragmentos do mesmo audio)**
- Cortar 2 fragmentos diferentes
- Calcular features e fingerprint
- Comparar por distancia e por descrición

### Parte B (reflexión, 8–10 liñas)
- Que feature che axudou máis a diferenciar?
- Coincide co que escoitas?
- Que limitacións ten esta aproximación (sen IA)?


In [ ]:
# ✅ Espazo de traballo: escolle a túa opción aquí

# Opción 1: carga dous audios teus (subidos) e chama fingerprint()
# ruta1 = "audio1.wav"
# ruta2 = "audio2.wav"

# Opción 2: usa un audio e corta dous fragmentos diferentes
# y, sr = librosa.load(ruta1, sr=None)
# frag1 = cortar(y, sr, 0, 5)
# frag2 = cortar(y, sr, 5, 10)
# fp1 = fingerprint(frag1, sr)
# fp2 = fingerprint(frag2, sr)
# print(dist(fp1, fp2), describir(fp1), describir(fp2))


## 🧠 Ler código como partitura (checkpoint)
Antes de entregar:
- `# CARGA`
- `# FEATURES`
- `# FINGERPRINT`
- `# COMPARACIÓN`
- `# INTERPRETACIÓN`
O notebook debe lerse como un “informe musical” estruturado.
